In [23]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


from langdetect import detect_langs
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [ ]:
#twitter api keys
auth = tweepy.OAuth1UserHandler(
   )

api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:
search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
search_query = search_words + " -filter:retweets AND -filter:replies"


limit = 100000
tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

data=[]
for tweet in tweets:
    data.append([tweet.id,tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count, tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [ ]:
columns =['id','username','time_of_tweet', 'tweet', 'location', 'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']
bp_tweets=pd.DataFrame(data, columns=columns)
bp_tweets.to_csv("BlackPanther-24-11.csv")

In [4]:
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
27702,27702,1593058712780148736,LEJerni13,2022-11-17 01:49:29+00:00,Uhm. Yeah. So I fully ugly cried through appro...,NaN,0,0,727,877,False,Twitter for iPhone
33223,33223,1592748164368699394,EIianaLuna,2022-11-16 05:15:28+00:00,Un wáas k’oop al que diga que Kukulcán es Azte...,NaN,0,1,484,596,False,Twitter for iPhone
65,65,1595736832956067841,abdulvhad,2022-11-24 11:11:22+00:00,Killmonger showing up in #WakandaForeverID was...,NaN,0,1,4608,3230,False,Twitter for Android
14428,14428,1594109767487836160,ms_mimetica,2022-11-19 23:26:00+00:00,Estamos de acuerdo en que Okoye con su estilaz...,NaN,0,0,7,319,False,Twitter for Android
13146,13146,1594186657309560832,notladyblu,2022-11-20 04:31:32+00:00,#WakandaForever My husband wanted to see it. I...,NaN,0,2,1690,2255,False,Twitter for Android
14849,14849,1594088136845688833,Cinepolis,2022-11-19 22:00:03+00:00,Los reales y yo llegando a ver #WakandaForever...,México,17,207,8163877,20721,True,Emplifi
22477,22477,1593431042450407425,toryhargro,2022-11-18 02:28:59+00:00,#Shuri has the most compelling hero’s journey ...,"Atlanta, GA",1,3,2657,2813,True,Twitter for iPhone
32005,32005,1592839814893957122,onntomorina,2022-11-16 11:19:40+00:00,พรุ่งนี้ day off ไปดู #WakandaForever อีกรอบดี...,NaN,0,3,2184,993,False,Twitter for Android
346,346,1595663039562498050,cinemaecritica,2022-11-24 06:18:09+00:00,Amsterdam: Il Culto della Libertà\nScopri l’Ot...,NaN,0,1,226,50,False,Twitter Web App
27988,27988,1593041673017511936,4lexandria__,2022-11-17 00:41:46+00:00,#WakandaForever had me ugly crying.,77(3)12,0,0,287,391,False,Twitter for iPhone


In [ ]:
tweets.info()

In [ ]:
tweets.head()

In [5]:
tweets.drop(columns='Unnamed: 0', inplace = True)

In [6]:
tweets['location'].fillna('None', inplace=True)

In [7]:
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [8]:
def get_hash(tweet):
    tweet = re.findall(r'\#\w+', tweet)
    return " ".join(tweet)

In [9]:
tweets['hashtags'] = tweets['tweet'].apply(get_hash)


In [10]:
hashtags = pd.Series(" ".join(tweets['hashtags']).split()).value_counts()
#hashtags.to_csv('hashtag_freq.csv')
hashtags[:10]

#WakandaForever                20990
#BlackPanther                   9316
#BlackPantherWakandaForever     5887
#wakandaforever                 1997
#Namor                          1746
#blackpanther                   1297
#Marvel                         1211
#BlackPanther2                   998
#WakandaForeverID                978
#MarvelStudios                   808
dtype: int64

In [11]:
def get_languages(tweet):
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [12]:
tweets['language'] = tweets['tweet'].apply(get_languages)

In [13]:
lang_code = pd.read_csv('language_codes.csv')
dict_lang_code = dict(zip(lang_code['alpha2'], lang_code['English']))
tweets['language'] = tweets['language'].replace(dict_lang_code)

In [14]:
tweets.language.value_counts()[:10]

English       24248
Spanish        3315
Danish         1472
Portuguese     1464
French          768
Indonesian      505
Japanese        470
Thai            470
Norwegian       386
Italian         326
Name: language, dtype: int64

In [15]:
def get_links(tweets):
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [16]:
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [17]:
tweets.contains_media.value_counts()

Yes    20872
No     14182
Name: contains_media, dtype: int64

In [18]:
cast = ['namor', 'shuri', 'ironheart', 'ramonda', 'namora', 
        'okoye', 'aneka', 'nakia', 'mbaku', 'attuma', 'everett', ]


def get_cast(tweets):
    tweets = tweets.lower()
    tokens = word_tokenize(tweets)
    cast_names = [name for name in tokens if name in cast]
    cast_names_title = [name.title() for name in cast_names]
    return " ".join(cast_names_title)

In [19]:
tweets['cast_name'] = tweets['tweet'].apply(get_cast)

In [20]:
names = pd.Series(" ".join(tweets['cast_name']).split()).value_counts()
# names.to_csv('names_freq.csv') ##save the generated dataframe
names[:10]

Namor        3953
Shuri        1985
Okoye         348
Namora        295
Ironheart     288
Nakia         220
Ramonda       216
Mbaku         202
Attuma        123
Aneka          53
dtype: int64

In [21]:
tweets['tweet'] = tweets['tweet'].replace('\n', ' ', regex=True)


In [37]:
sid = SentimentIntensityAnalyzer()
tweets['sentient_scores'] = tweets['tweet'].apply(sid.polarity_scores)
tweets['sentient_scores'] = tweets['sentient_scores'].apply(lambda x: x['compound'])

In [39]:
tweets.sample(20)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,hashtags,language,contains_media,cast_name,sentient_scores
5684,1594865525481521154,SOPHI3_RVKH,2022-11-22 01:29:07,"#WakandaForeverID Recomendable, bien por #T...",Earth 🌎,0,10,3294,3028,False,Twitter for Android,#WakandaForeverID #TenochHuerta,Spanish,Yes,,0.0000
4572,1595060671758176256,Julhynha,2022-11-22 14:24:33,Quando estreia #BlackPanther na Disney?,Sorocaba/SP,0,0,899,749,False,Twitter for iPhone,#BlackPanther,English,No,,0.0000
27629,1593061709392285696,torrayyyyy_,2022-11-17 02:01:23,I wish some folks would save the awful “center...,In God's Hands,1,2,726,759,False,Twitter for iPhone,#WakandaForever,English,No,,0.8402
10718,1594399029458309123,PantherKing14,2022-11-20 18:35:25,THE BLACK PANTHER LIVES #WakandaForever https:...,"Spring, TX",0,2,436,2195,False,Twitter for iPhone,#WakandaForever,English,Yes,,0.0000
559,1595623080948555776,numeroboxoffice,2022-11-24 03:39:22,Weekly #BoxOffice Est. - Australia 1. #BlackP...,"Sydney, Australia",3,22,2480,369,False,Twitter Web App,#BoxOffice #BlackPanther #WakandaForever #MrsH...,English,Yes,,0.0000
19190,1593736442034913283,nerdishjournal,2022-11-18 22:42:32,Arte conceitual oficial do LINDO TRAJE de Pant...,"Rio de Janeiro, Brazil",1,19,687,736,False,Twitter for Android,#Shuri #WakandaForever,Portuguese,Yes,Shuri,-0.3595
33515,1592734763064123392,SM_Cinema,2022-11-16 04:22:13,📣 Catch 𝗠𝗮𝗿𝘃𝗲𝗹 𝗦𝘁𝘂𝗱𝗶𝗼𝘀' #BlackPanther: #Wakand...,The Philippines,0,0,83737,128,True,Twitter for iPhone,#BlackPanther #WakandaForever #1 #WakandaForev...,English,Yes,,0.0000
11052,1594385168462221315,ChioSmiles,2022-11-20 17:40:21,#BlackPanther 🇲🇽🤝 🇺🇸 Cultura 🔥,None,0,1,7501,2471,False,Twitter for iPhone,#BlackPanther,English,No,,0.0000
16979,1593945690111184897,Osazonline,2022-11-19 12:34:01,E no get wetin una won tell me una edit this p...,🇳🇬,0,2,17,72,False,Twitter for Android,#WakandaForever,English,Yes,,0.4215
14511,1594104741323108352,gary_rose,2022-11-19 23:06:02,#WakandaForever YOU GET AN IRON MAN SUIT! Y...,Gateshead/Durham,0,0,1227,792,False,Twitter for iPhone,#WakandaForever,English,Yes,,0.0000


In [ ]:
cv = tweets['location'].replace(np.nan, "None")

In [ ]:
cv[cv.str.contains('IL')][:60]
